In [1]:
!unzip emojIQ_deepseekmath-r1.zip

Archive:  emojIQ_deepseekmath-r1.zip
   creating: emojIQ/
  inflating: emojIQ/adapter_model.safetensors  
  inflating: emojIQ/special_tokens_map.json  
   creating: emojIQ/checkpoint-294/
  inflating: emojIQ/checkpoint-294/adapter_model.safetensors  
  inflating: emojIQ/checkpoint-294/README.md  
  inflating: emojIQ/checkpoint-294/optimizer.pt  
  inflating: emojIQ/checkpoint-294/adapter_config.json  
  inflating: emojIQ/checkpoint-294/scheduler.pt  
  inflating: emojIQ/checkpoint-294/rng_state.pth  
  inflating: emojIQ/checkpoint-294/trainer_state.json  
  inflating: emojIQ/checkpoint-294/training_args.bin  
  inflating: emojIQ/README.md        
  inflating: emojIQ/tokenizer_config.json  
   creating: emojIQ/checkpoint-300/
  inflating: emojIQ/checkpoint-300/adapter_model.safetensors  
  inflating: emojIQ/checkpoint-300/README.md  
  inflating: emojIQ/checkpoint-300/optimizer.pt  
  inflating: emojIQ/checkpoint-300/adapter_config.json  
  inflating: emojIQ/checkpoint-300/scheduler.pt 

In [2]:
!pip install transformers
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 3.5 MB/s eta 0:00:00


In [3]:
from transformers import AutoModelForCausalLM
from peft import PeftModel

model = AutoModelForCausalLM.from_pretrained("deepseek-ai/deepseek-math-7b-rl")
model = PeftModel.from_pretrained(model, "./emojIQ")

model = model.merge_and_unload()
model.save_pretrained("emojiIQ_deepseekmath-r1")

/usr/local/lib/python3.11/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.8k [00:00<?, ?B/s]

model-00001-of-000002.safetensors:   0%|          | 0.00/8.59G [00:00<?, ?B/s]

model-00002-of-000002.safetensors:   0%|          | 0.00/5.23G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

In [ ]:
!zip -r emojIQ_deepseekmath-r1_model.zip ./emojiIQ_deepseekmath-r1

  adding: emojiIQ_deepseekmath-r1/ (stored 0%)
  adding: emojiIQ_deepseekmath-r1/model-00001-of-00006.safetensors

In [ ]:
!pip install kagglehub

In [ ]:
import kagglehub

kagglehub.login()

In [ ]:
# Replace with path to directory containing model files.
LOCAL_MODEL_DIR = 'emojIQ_deepseekmath-r1_model.zip'

kagglehub.model_upload(
  handle = 'buzzgrewal/emojiq/transformers/default',
  local_model_dir = LOCAL_MODEL_DIR,
  version_notes = 'Update 2025-03-07')

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


base_model_name = "deepseek-ai/deepseek-math-7b-rl"
adapter_model_name = "./emojIQ"
merged_model_path = "./emojIQ_deepseekmath-r1"


tokenizer = AutoTokenizer.from_pretrained(adapter_model_name)


device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(merged_model_path).to(device)


test_cases = [
    "🍕 + 🍕 + 🍕 = 21",
    "🚗 + 🚗 + 🚗 + 🚗 = 36",
    "🐱 + 🐱 = 14",
    "🎸 + 🎸 + 🎸 + 🎸 + 🎸 = 50",
    "🌞 + 🌞 = 20",
    "🦄 + 🦄 + 🦄 = 27",
    "🎁 + 🎁 + 🎁 + 🎁 = 32",
    "📚 + 📚 = 18",
    "⚽ + ⚽ + ⚽ + ⚽ + ⚽ = 45",
    "🍩 + 🍩 = 16"
]

def correct_math_result(math_statement):
    prompt = f"Incorrect: {math_statement}\nCorrect:"

    inputs = tokenizer(prompt, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=100, temperature=0.7, do_sample=True)

    corrected_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected_output.split("Correct:")[-1].strip()



print("🔍 Testing Fine-Tuned Model on Incorrect Math Memes:\n")
for i, case in enumerate(test_cases, 1):
    corrected = correct_math_result(case)
    print(f"❌ Incorrect: {case}\n✅ Model's Correction: {corrected}\n{'-'*50}")
